Task1_0725. 타이타닉 생존자 예측 데이터 세트 train.csv에 대하여 다음 사항을 수행하세요.
- 일괄 전처리 사용자 함수 transform_features(df) 작성
- 분류 모델 학습 및 평가 사용자 함수 작성
- dt, lr, rf 모델링 및 평가

- GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행.
  - Decision Tree, Random Forest, Logistic Regression 모델별 수행
  - 선택한 모델에 적합한 parameter greed 적용
  - cv=5 적용

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt

df = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def transform_features(df):
    # 결측값 처리
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Fare'].fillna(df['Fare'].mean(), inplace=True)

    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

    df.drop(['Name', 'Ticket', 'Cabin', 'Embarked','PassengerId'], axis=1, inplace=True)
    return df

df_processed = transform_features(df)

# 전역 변수로 모델 정의
dt_model = DecisionTreeClassifier(random_state=42)
lr_model = LogisticRegression(random_state=42)
rf_model = RandomForestClassifier(random_state=42)

def train_and_evaluate(df_processed):
    X = df_processed.drop('Survived', axis=1)
    y = df_processed['Survived']

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    global dt_model, lr_model, rf_model  # 전역 변수로 접근할 수 있도록 선언

    dt_model.fit(X_train, y_train)
    lr_model.fit(X_train, y_train)
    rf_model.fit(X_train, y_train)

    models = {'DecisionTree' : dt_model,
              'LogisticRegression': lr_model,
              'RandomForest': rf_model}

    for model_name, model in models.items():
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:,1]

        confusion = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)

        print(f'모델: {model_name}')
        print(f'오차행렬:\n{confusion}')
        print(f'정확도: {accuracy:.4f}')
        print(f'정밀도: {precision:.4f}')
        print(f'재현율: {recall:.4f}')
        print(f'F1 Score: {f1:.4f}')
        print(f'ROC AUC Score: {roc_auc:.4f}')
        print('')

    return X_train, X_test, y_train, y_test


# train_and_evaluate 함수 호출
X_train, X_test, y_train, y_test = train_and_evaluate(df_processed)

# GridSearchCV를 이용한 최적 모델 찾기
parameters = {'max_depth':[2,3,5,10,12],
             'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8,10]}

grid_dclf = GridSearchCV(dt_model , param_grid=parameters , scoring='accuracy' , cv=5)
grid_dclf.fit(X_train , y_train)

print('GridSearchCV 최적 하이퍼 파라미터 :',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

# 최적 모델을 이용한 평가
y_pred = best_dclf.predict(X_test)
y_pred_proba = best_dclf.predict_proba(X_test)[:,1]

confusion = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)


print(f'오차행렬:\n{confusion}')
print(f'정확도: {accuracy:.4f}')
print(f'정밀도: {precision:.4f}')
print(f'재현율: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC Score: {roc_auc:.4f}')
print('')


모델: DecisionTree
오차행렬:
[[85 20]
 [24 50]]
정확도: 0.7542
정밀도: 0.7143
재현율: 0.6757
F1 Score: 0.6944
ROC AUC Score: 0.7514

모델: LogisticRegression
오차행렬:
[[92 13]
 [21 53]]
정확도: 0.8101
정밀도: 0.8030
재현율: 0.7162
F1 Score: 0.7571
ROC AUC Score: 0.8802

모델: RandomForest
오차행렬:
[[91 14]
 [22 52]]
정확도: 0.7989
정밀도: 0.7879
재현율: 0.7027
F1 Score: 0.7429
ROC AUC Score: 0.8816

GridSearchCV 최적 하이퍼 파라미터 : {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.8244
오차행렬:
[[92 13]
 [23 51]]
정확도: 0.7989
정밀도: 0.7969
재현율: 0.6892
F1 Score: 0.7391
ROC AUC Score: 0.8456

